# Code Description:

In the following code, the closing price for each sector is represented by the average of the adjusted closing   prices of the constituent companies within that sector. Here's a breakdown of the steps involved in calculating this average closing price for each sector:

#### Data Collection:
- The code first fetches the list of S&P 500 companies and their respective sectors using the Wikipedia link provided.
- It then replaces specific tickers like 'BRK.B' with 'BRK-B' and 'BF.B' with 'BF-B' to ensure compatibility with Yahoo Finance.

#### Looping Through Sectors:
- The code then loops through each unique sector in the dataset.

#### Fetching Company Data:
- For each sector, it retrieves the adjusted closing prices of the constituent companies for the last 90 days from Yahoo Finance.

#### Calculating Average Closing Price:
It calculates the average closing price for each date by taking the mean of the adjusted closing prices for all companies within that sector on that date.

#### Data Formatting and Organization:
- The code organizes this data into a DataFrame with columns for 'Date', 'Sector', and 'Avg_Close' (average closing price).
- The DataFrame is pivoted to have sectors as columns and dates as rows.
- Finally, the DataFrame is stacked to have the sectors' average closing prices in a single column for each date.

#### Data Saving:
- The resulting DataFrame, representing the average closing prices for each sector, is saved to an Excel file.
- The calculated average closing prices for each sector represent an aggregate view of the performance of companies within that sector, giving an indication of the overall sector's stock price movement over the specified period (last 90 days in this case).

In [ ]:
#installs necessary libraries
%pip install openpyxl
%pip install pandas
%pip install yfinance
%pip install matplotlib
%pip install scipy
%pip install scikit-learn
%pip install PyPortfolioOpt
%pip install cvxpy

In [ ]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import pathlib
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as sci_opt

from pprint import pprint
from sklearn.preprocessing import StandardScaler
# Set some display options for Pandas.
%config InlineBackend.figure_format ='retina'
pd.set_option('display.max_colwidth', None)
pd.set_option('expand_frame_repr', False)
pd.set_option('display.max_columns', None)



In [ ]:
## Fetching S&P 500 Companies:
sp500_companies = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500_companies['Symbol'] = sp500_companies['Symbol'].replace({'BRK.B': 'BRK-B', 'BF.B': 'BF-B'})

In [ ]:
sp500_companies

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...
498,XYL,Xylem Inc.,Industrials,Industrial Machinery & Supplies & Components,"White Plains, New York",2011-11-01,1524472,2011
499,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
500,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
501,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927


In [ ]:
# Dropping NaN values
sp500_companies.dropna(inplace=True)

In [ ]:
## Getting Unique Sectors and Dates:
unique_sectors = sp500_companies['GICS Sector'].unique()
end_date = datetime.today().strftime('%Y-%m-%d')
start_date = (datetime.today() - timedelta(days=90)).strftime('%Y-%m-%d')

In [ ]:
unique_sectors_df = pd.DataFrame(unique_sectors)
print(unique_sectors_df)

                         0
0              Industrials
1              Health Care
2   Information Technology
3                Utilities
4               Financials
5                Materials
6   Consumer Discretionary
7              Real Estate
8   Communication Services
9         Consumer Staples
10                  Energy


In [ ]:
number_of_sectors = len(unique_sectors)

In [ ]:
len(unique_sectors)

11

In [ ]:
## Fetching Historical Prices and Calculating Averages:
## The code fetches historical closing prices for each company in each sector and calculates the average closing price for the sector.
# Loop through each sector and download closing prices
all_avg_prices = []
all_dates = []
all_sectors = []
for sector in unique_sectors:
    companies = sp500_companies[sp500_companies['GICS Sector'] == sector]['Symbol'].tolist()
    prices = yf.download(companies, start=start_date, end=end_date)['Adj Close']
    avg_prices = prices.mean(axis=1)
    # Append the data
    all_avg_prices.extend(avg_prices.tolist())
    all_dates.extend([date.date() for date in prices.index])
    all_sectors.extend([sector] * len(avg_prices))

# Create a DataFrame
sp500_sectors_avg_prices = pd.DataFrame({
    'Date': all_dates,
    'Avg_Close': all_avg_prices,
    'Sector': all_sectors
})

#sp500_sectors_avg_prices = sp500_sectors_avg_prices.pivot(index='Date', columns='Sector', values='Avg_Close')
# Reverse the order of the DataFrame by dates
sp500_sectors_avg_prices = sp500_sectors_avg_prices[::-1]

print(sp500_sectors_avg_prices.head(100))

[*********************100%***********************]  78 of 78 completed
[*********************100%***********************]  63 of 63 completed
[*********************100%***********************]  67 of 67 completed
[*********************100%***********************]  31 of 31 completed
[*********************100%***********************]  71 of 71 completed
[*********************100%***********************]  28 of 28 completed
[*********************100%***********************]  52 of 52 completed
[*********************100%***********************]  31 of 31 completed
[*********************100%***********************]  22 of 22 completed
[*********************100%***********************]  38 of 38 completed
[*********************100%***********************]  22 of 22 completed


           Date   Avg_Close            Sector
681  2024-08-26   87.699545            Energy
680  2024-08-23   87.243636            Energy
679  2024-08-22   85.905000            Energy
678  2024-08-21   85.827728            Energy
677  2024-08-20   85.497955            Energy
..          ...         ...               ...
586  2024-07-10  105.777761  Consumer Staples
585  2024-07-09  105.365021  Consumer Staples
584  2024-07-08  105.778489  Consumer Staples
583  2024-07-05  105.948899  Consumer Staples
582  2024-07-03  104.701393  Consumer Staples

[100 rows x 3 columns]


In [ ]:
sp500_sectors_avg_prices = sp500_sectors_avg_prices.pivot(
    index='Date',
    columns='Sector',
    values='Avg_Close'
)
print(sp500_sectors_avg_prices.head())

Sector      Communication Services  Consumer Discretionary  Consumer Staples     Energy  Financials  Health Care  Industrials  Information Technology   Materials  Real Estate  Utilities
Date                                                                                                                                                                                     
2024-05-29              126.753480              427.194581        104.201668  86.518504  164.710151   240.404132   208.768401              263.792091  147.129167   118.829592  72.521968
2024-05-30              126.259379              432.292042        104.317818  86.742961  165.345931   239.600468   210.214910              257.621378  148.473282   120.349102  73.529537
2024-05-31              127.278646              439.142145        106.150326  88.727149  167.347147   242.702178   212.586944              257.171589  150.135157   122.290814  74.618398
2024-06-03              127.397392              437.348366        106.

In [ ]:
sp500_sectors_avg_prices.dropna(inplace=True)

In [ ]:
print(sp500_sectors_avg_prices.isnull().sum())

Sector
Communication Services    0
Consumer Discretionary    0
Consumer Staples          0
Energy                    0
Financials                0
Health Care               0
Industrials               0
Information Technology    0
Materials                 0
Real Estate               0
Utilities                 0
dtype: int64


In [ ]:
sp500_sectors_avg_prices.to_excel('sp500_sectors_avg_prices.xlsx', index=True)

In [ ]:
# Calculate the Log of returns.
log_return = np.log(1 +sp500_sectors_avg_prices.pct_change().iloc[::-1])

# Drop rows with negative values
log_return = log_return[(log_return > 0).all(axis=1)]


# Generate Random Weights.
random_weights = np.array(np.random.random(number_of_sectors))

# Generate the Rebalance Weights, these should equal 1.
rebalance_weights = random_weights / np.sum(random_weights)


In [ ]:
print('Log Returns:')
print(log_return.head())

Log Returns:
Sector      Communication Services  Consumer Discretionary  Consumer Staples    Energy  Financials  Health Care  Industrials  Information Technology  Materials  Real Estate  Utilities
Date                                                                                                                                                                                   
2024-08-23                0.005366                0.020280          0.003841  0.015463    0.008392     0.006138     0.011847                0.009668   0.014703     0.019247   0.002602
2024-08-21                0.003894                0.012672          0.008309  0.003850    0.000790     0.002300     0.007834                0.010537   0.012521     0.002181   0.007539
2024-08-19                0.011836                0.012624          0.004600  0.005294    0.005251     0.006691     0.006168                0.007152   0.003057     0.004946   0.006350
2024-08-08                0.025052                0.021700         

In [ ]:
print((sp500_sectors_avg_prices == 0).sum())

Sector
Communication Services    0
Consumer Discretionary    0
Consumer Staples          0
Energy                    0
Financials                0
Health Care               0
Industrials               0
Information Technology    0
Materials                 0
Real Estate               0
Utilities                 0
dtype: int64


In [ ]:
log_return.to_excel('log_return.xlsx', index=True)

In [ ]:
# Calculate the percentage of negative log returns
percentage_negative_returns = (log_return < 0).mean().mean() * 100

print(f"Percentage of negative log returns: {percentage_negative_returns:.2f}%")

Percentage of negative log returns: 0.00%


In [ ]:

# Calculate the Expected Returns, annualize it by multiplying it by `252`.
risk_free_rate = .01
exp_ret = np.sum(((log_return.mean()-risk_free_rate) * rebalance_weights) * 252)

# Calculate the Expected Volatility, annualize it by multiplying it by `252`.
exp_vol = np.sqrt(
np.dot(
    rebalance_weights.T,
    np.dot(
        log_return.cov() * 252,
        rebalance_weights
    )
)
)

# Calculate the Sharpe Ratio.
sharpe_ratio = exp_ret / exp_vol

# Put the weights into a data frame to see them better.
weights_df = pd.DataFrame(data={
'random_weights': random_weights,
'rebalance_weights': rebalance_weights
})
print('')
print('='*80)
print('PORTFOLIO WEIGHTS:')
print('-'*80)
print(weights_df)
print('-'*80)

# Do the same with the other metrics.
metrics_df = pd.DataFrame(data={
    'Expected Portfolio Returns': exp_ret,
    'Expected Portfolio Volatility': exp_vol,
    'Portfolio Sharpe Ratio': sharpe_ratio
}, index=[0])

print('')
print('='*80)
print('PORTFOLIO METRICS:')
print('-'*80)
print(metrics_df)
print('-'*80)


PORTFOLIO WEIGHTS:
--------------------------------------------------------------------------------
    random_weights  rebalance_weights
0         0.999391           0.184981
1         0.785392           0.145371
2         0.553556           0.102459
3         0.138209           0.025581
4         0.128185           0.023726
5         0.721611           0.133565
6         0.956812           0.177099
7         0.066885           0.012380
8         0.298274           0.055209
9         0.154541           0.028604
10        0.599826           0.111024
--------------------------------------------------------------------------------

PORTFOLIO METRICS:
--------------------------------------------------------------------------------
   Expected Portfolio Returns  Expected Portfolio Volatility  Portfolio Sharpe Ratio
0                    0.389642                       0.073222                5.321408
--------------------------------------------------------------------------------


In [ ]:
# Initialize the components, to run a Monte Carlo Simulation.

# We will run 5000 iterations.
num_of_portfolios = 20000

# Prep an array to store the weights as they are generated, 5000 iterations for each of our 4 symbols.
all_weights = np.zeros((num_of_portfolios, number_of_sectors))

# Prep an array to store the returns as they are generated, 5000 possible return values.
ret_arr = np.zeros(num_of_portfolios)

# Prep an array to store the volatilities as they are generated, 5000 possible volatility values.
vol_arr = np.zeros(num_of_portfolios)

# Prep an array to store the sharpe ratios as they are generated, 5000 possible Sharpe Ratios.
sharpe_arr = np.zeros(num_of_portfolios)

# Start the simulations.
for ind in range(num_of_portfolios):

    # First, calculate the weights.
    weights = np.array(np.random.random(number_of_sectors))
    weights = weights / np.sum(weights)

    # Add the weights, to the `weights_arrays`.
    all_weights[ind, :] = weights

    # Calculate the expected log returns, and add them to the `returns_array`.
    ret_arr[ind] = np.sum(((log_return.mean()-risk_free_rate) * weights) * 252)

    # Calculate the volatility, and add them to the `volatility_array`.
    vol_arr[ind] = np.sqrt(
        np.dot(weights.T, np.dot(log_return.cov() * 252, weights)))


    # Calculate the Sharpe Ratio and Add it to the `sharpe_ratio_array`.
    sharpe_arr[ind] = ret_arr[ind]/vol_arr[ind]

# Let's create our "Master Data Frame", with the weights, the returns, the volatility, and the Sharpe Ratio
simulations_data = [ret_arr, vol_arr, sharpe_arr, all_weights]

# Create a DataFrame from it, then Transpose it so it looks like our original one.
simulations_df = pd.DataFrame(data=simulations_data).T

# Give the columns the Proper Names.
simulations_df.columns = [
    'Returns',
    'Volatility',
    'Sharpe Ratio',
    'Portfolio Weights'
]

# Make sure the data types are correct, we don't want our floats to be strings.
simulations_df = simulations_df.infer_objects()

# Print out the results.
print('')
print('='*80)
print('SIMULATIONS RESULT:')
print('-'*80)

# Print PORTFOLIO WEIGHTS
print('='*80)
print('PORTFOLIO WEIGHTS:')
print('-'*80)
print(simulations_df['Portfolio Weights'].head())
print('-'*80)

# Print PORTFOLIO METRICS
print('='*80)
print('PORTFOLIO METRICS:')
print('-'*80)
print(simulations_df[['Returns', 'Volatility', 'Sharpe Ratio']].head())
print('-'*80)


SIMULATIONS RESULT:
--------------------------------------------------------------------------------
PORTFOLIO WEIGHTS:
--------------------------------------------------------------------------------
0     [0.012326686401708316, 0.12822271985859796, 0.15095696542100873, 0.14204242930167968, 0.002789737121416752, 0.1374485499941389, 0.12484669624133797, 0.0331928982419909, 0.14343048211044718, 0.08810756590477177, 0.036635269402901766]
1      [0.06431792451160978, 0.06519417511340624, 0.10929976701402958, 0.09602028839739718, 0.1505689176279073, 0.1381427900513829, 0.15071242618141656, 0.060871217182538316, 0.05619650627529693, 0.003414592789360838, 0.10526139485565436]
2       [0.18383993978115004, 0.04736211572674533, 0.10603709569604444, 0.0457973179122562, 0.050110194625459376, 0.0229349631878563, 0.15469266291128844, 0.10985642448702447, 0.12651514625498647, 0.02651505284371792, 0.12633908657347104]
3        [0.14744898167197734, 0.0761388181436209, 0.04382969277729056, 0.0528686

In [ ]:
# Return the Max Sharpe Ratio from the run.
max_sharpe_ratio = simulations_df.loc[simulations_df['Sharpe Ratio'].idxmax()]

# Return the Min Volatility from the run.
min_volatility = simulations_df.loc[simulations_df['Volatility'].idxmin()]

print('')
print('='*80)
print('MAX SHARPE RATIO:')
print('-'*80)
print(max_sharpe_ratio)
print('-'*80)

print('')
print('='*80)
print('MIN VOLATILITY:')
print('-'*80)
print(min_volatility)
print('-'*80)



MAX SHARPE RATIO:
--------------------------------------------------------------------------------
Returns                                                                                                                                                                                                                                                 0.762855
Volatility                                                                                                                                                                                                                                              0.073679
Sharpe Ratio                                                                                                                                                                                                                                           10.353813
Portfolio Weights    [0.03979404505777936, 0.2516083461931149, 0.03149998754651055, 0.029825881913984217, 0.019102389116181367, 0

### Portfolio Weights:

The portfolio is again diversified, with the highest weight (26.83%) on the 1st asset and significant weights on the 3rd and 11th assets.
The weights are determined to minimize the volatility of the portfolio, resulting in a more conservative allocation.

In summary, the first portfolio aims to maximize the risk-adjusted return (Sharpe Ratio) and has a higher expected return but also higher volatility. The second portfolio aims to minimize volatility with a more conservative allocation but with a lower expected return. The choice between the two depends on the investor's risk tolerance and investment objectives.

### Portfolio Weights:

#### Max Sharpe Ratio Portfolio:

#### Sector Weights:
- Sector 1: 18.10%
- Sector 2: 4.58%
- Sector 3: 1.05%
- Sector 4: 4.37%
- Sector 5: 0.43%
- Sector 6: 2.78%
- Sector 7: 6.19%
- Sector 8: 22.73%
- Sector 9: 14.14%
- Sector 10: 20.24%
- Sector 11: 5.39%

#### Min Volatility Portfolio:

#### Sector Weights:
- Sector 1: 26.83%
- Sector 2: 2.88%
- Sector 3: 23.50%
- Sector 4: 5.97%
- Sector 5: 0.70%
- Sector 6: 7.63%
- Sector 7: 4.40%
- Sector 8: 3.56%
- Sector 9: 1.41%
- Sector 10: 3.40%
- Sector 11: 19.73%

We can further these results with machine learning via the scikit library

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


simulations_df["Portfolio Weights"] = simulations_df["Portfolio Weights"].apply(lambda x: np.array(x) if isinstance(x, list) else x)


flattened_weights = np.vstack(simulations_df["Portfolio Weights"].to_numpy())


X = simulations_df[["Volatility", "Sharpe Ratio"]].values
y = flattened_weights


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


rf_models = []
y_preds = []


for i in range(y.shape[1]):
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
    rf_model.fit(X_train, y_train[:, i])
    y_pred = rf_model.predict(X_test)

    rf_models.append(rf_model)
    y_preds.append(y_pred)


y_preds = np.array(y_preds).T


print("Predicted Portfolio Weights (first 5 samples):")
print(y_preds[:5])

print("\nActual Portfolio Weights (first 5 samples):")
print(y_test[:5])

mse_list = [mean_squared_error(y_test[:, i], y_preds[:, i]) for i in range(y.shape[1])]

print("\nMean Squared Errors for each portfolio weight:")
print(mse_list)

Predicted Portfolio Weights (first 5 samples):
[[0.11865427 0.09900773 0.0868748  0.08033465 0.07555186 0.08339413
  0.08187804 0.08771116 0.11153629 0.10560091 0.07402798]
 [0.07641455 0.10241499 0.09298588 0.10220002 0.05324215 0.09310165
  0.11552621 0.10641491 0.08572952 0.08384608 0.10048157]
 [0.06802732 0.10098549 0.1071878  0.09838886 0.10277012 0.07738008
  0.07033473 0.06052111 0.1072255  0.08182558 0.09682677]
 [0.09393774 0.12167835 0.04064455 0.12151531 0.1106865  0.1003432
  0.13532038 0.1165107  0.1092591  0.05559389 0.02144142]
 [0.06256268 0.0872198  0.13156066 0.07994009 0.07960389 0.07300267
  0.06466923 0.08100808 0.06225375 0.13365622 0.13660312]]

Actual Portfolio Weights (first 5 samples):
[[0.09487963 0.03685253 0.04935337 0.0688838  0.1539331  0.07153798
  0.14574185 0.00493499 0.17736134 0.16455561 0.0319658 ]
 [0.0268248  0.06938469 0.14685774 0.18618465 0.088873   0.0168965
  0.2064027  0.00984455 0.18802252 0.0562489  0.00445995]
 [0.01470487 0.04202669 0.0